In [41]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.regression import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from dbn.tensorflow import SupervisedDBNRegression

In [42]:
# Read the dataset
ROAD = "Taft Ave."
YEAR = "2015"
EXT = ".csv"
WINDOWSIZE = 1
FILENAME = "eng_win" + str(WINDOWSIZE) + "_mmda_" + ROAD + "_" + YEAR
#FILENAME = "noeng" + "_mmda_" + ROAD + "_" + YEAR

In [43]:
original_dataset = pd.read_csv("data/mmda/" + FILENAME + EXT, skipinitialspace=True)
original_dataset = original_dataset.fillna(0)

##### Preparing Traffic Dataset

In [44]:
original_dataset = pd.read_csv("data/mmda/" + FILENAME + EXT, skipinitialspace=True)
original_dataset = original_dataset.fillna(0)
traffic_dataset = original_dataset
#print("Start : " + str(original_dataset.columns[0:original_dataset.shape[1]][5]))
#print("End : " + str(original_dataset.columns[0:original_dataset.shape[1]][traffic_dataset.shape[1]-1]))

# Remove date time. Remove unused columms
#0-2 = dt + lineName + stationName || 3-4 - statusN - statusS || 5-14 - original weather variables
#15-46 - engineered traffic
cols_to_remove = [0, 1, 2]

traffic_dataset.drop(traffic_dataset.columns[[cols_to_remove]], axis=1, inplace=True)
traffic_dataset.head()

,statusN,statusS,statusN_Emean,statusN_Emin,statusN_Emax,statusN_Esum,statusS_Emean,statusS_Emin,statusS_Emax,statusS_Esum
0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,0.5,0.5,0.5,0.5,0.5,1.0,0.5,0.5,0.5,1.0
2,0.5,0.5,0.5,0.5,0.5,1.5,0.5,0.5,0.5,1.5
3,0.5,0.5,0.5,0.5,0.5,2.0,0.5,0.5,0.5,2.0
4,0.5,0.5,0.5,0.5,0.5,2.5,0.5,0.5,0.5,2.5


In [45]:
# To-be Predicted variable 
Y = traffic_dataset.statusS
Y = Y.shift(-3)
Y = Y.fillna(0)
Y = Y.round(5)

# Other data
X = traffic_dataset
#X = dataset
#X.statusS = X.statusS.round(5)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.67, shuffle=False)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# Data scaling
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)

In [46]:
# Training
regressor = SupervisedDBNRegression(hidden_layers_structure=[5, 10, 15],
                                    learning_rate_rbm=0.01,
                                    learning_rate=0.01,
                                    n_epochs_rbm=3,
                                    n_iter_backprop=5,
                                    batch_size=16,
                                    activation_function='relu')
regressor.fit(X_train, Y_train)


[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.393830
>> Epoch 2 finished 	RBM Reconstruction error 0.290099
>> Epoch 3 finished 	RBM Reconstruction error 0.237003
>> Epoch 1 finished 	RBM Reconstruction error 0.132969
>> Epoch 2 finished 	RBM Reconstruction error 0.093690
>> Epoch 3 finished 	RBM Reconstruction error 0.078197
>> Epoch 1 finished 	RBM Reconstruction error 0.070729
>> Epoch 2 finished 	RBM Reconstruction error 0.052666
>> Epoch 3 finished 	RBM Reconstruction error 0.042424
[END] Pre-training step
[START] Fine tuning step:
>> Epoch 0 finished 	ANN training loss 0.109394
>> Epoch 1 finished 	ANN training loss 0.078847
>> Epoch 2 finished 	ANN training loss 0.033290
>> Epoch 3 finished 	ANN training loss 0.027303
>> Epoch 4 finished 	ANN training loss 0.026716
[END] Fine tuning step


SupervisedDBNRegression(batch_size=16, dropout_p=0, l2_regularization=1.0,
            learning_rate=0.01, n_iter_backprop=5, verbose=True)

In [47]:
# Test
X_test = min_max_scaler.transform(X_test)
Y_pred = regressor.predict(X_test)
print('Done.\nR-squared: %f\nMSE: %f' % (r2_score(Y_test, Y_pred), mean_squared_error(Y_test, Y_pred)))


Done.
R-squared: 0.678830
MSE: 0.037662


In [48]:
print(len(Y_pred))
temp = []
for i in range(len(Y_pred)):
    temp.append(Y_pred[i][0])
d = {'Predicted': temp, 'Actual': Y_test}

df = pd.DataFrame(data=d)
df.head()

23477


,Actual,Predicted
0,1.0,0.966272
1,1.0,0.966276
2,0.5,0.966279
3,0.5,0.966283
4,0.5,0.966287


In [49]:
df

,Actual,Predicted
0,1.0,0.966272
1,1.0,0.966276
2,0.5,0.966279
3,0.5,0.966283
4,0.5,0.966287
5,0.5,0.514982
6,0.5,0.514982
7,0.5,0.514981
8,0.5,0.514980
9,0.5,0.514979


In [50]:
# Save the model
regressor.save('models/pm1-witheng.pkl')

# # Restore
# classifier = SupervisedDBNClassification.load('model.pkl')

# # Test
# Y_pred = classifier.predict(X_test)
# print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))


In [51]:
df.to_csv("output/pm1_eng_output_" + FILENAME + EXT, encoding='utf-8')